# Standart STL containers and C++ features
https://en.cppreference.com/w/cpp/container
## Sequences (arrays/linked lists): ordered collections
### std::vector

Used features:
- placement new
- emplace
- templates
- random
- iota
- lambda functions
- initializer_list

In [1]:
#include <iostream>
#include <vector>
#include <cstdlib>
#include <ctime>
#include <new>
#include <numeric>
#include <initializer_list>

using std::vector;
using std::cout;
using std::cin;
using std::endl;
using std::rand;

class PlacementExample {
    int val;
public:
    PlacementExample (int x): val(x) {}
    operator int () const { return val; }
    ~PlacementExample () {}
};

template <class T>
void print (const vector <T> &a) {
    for (unsigned i = 0; i < a.size(); ++i) {
        cout << int(a[i]);
        if (i != a.size() - 1) {
            cout << ' ';
        } else {
            cout << endl;
        }
    }
}

int main (void) {
    vector <int> a;
    std::srand(std::time(0));
    for (int i = 0; i < 10; ++i) {
        a.push_back(i);
    }
    cout << endl << endl << "Sample vector: " << endl;
    cout << "a: ";
    print<int>(a);
    a.clear(); // Deletes all elements, sets size to nil, leaves capacity as it has been.
    cout << "CLEAR: ";
    cout << "Size: " << a.size() << " Capacity: " << a.capacity() << endl; // 0 and 16 respectively

    cout << "Is container empty: " << a.empty() << endl; // Checks if size is nil.

    cout << "Max size: " << a.max_size() << endl; // Returns the theoretical limit of container size in system

    a.reserve(5); // No effect, since capacity is more then value requested.
    cout << "RESRV  5: ";
    cout << "Size: " << a.size() << " Capacity: " << a.capacity() << endl; // 0 and 16 respectively

    a.reserve(18); // New block of memory is allocated
    cout << "RESRV 18: ";
    cout << "Size: " << a.size() << " Capacity: " << a.capacity() << endl; // 0 and 18 respectively

    a.shrink_to_fit(); // Request to free unused memory (not always fulfilled)
    cout << "SHRNK: ";
    cout << "Size: " << a.size() << " Capacity: " << a.capacity() << endl; // 0 and 18 respectively

    cout << endl << endl << "Generating new vector: " << endl;
    unsigned sz = std::abs(rand()) % 30;
    for (unsigned i = 0; i < sz; ++i) {
        a.push_back(rand() % 100);
    }
    print<int>(a);
    cout << "Size: " << a.size() << " Capacity: " << a.capacity() << endl;

    auto it = a.insert(a.end(), -10);
    cout << "INSRT: ";
    print<int>(a);
    cout << "INSRT: ";
    a.insert(it, 2, 300);
    print<int>(a);
    cout << "INSRT: ";
    a.insert(a.end(), {50, 51, 73});
    print<int>(a);

    cout << endl << endl << "Printing new allocated array using placement-new:" << endl;
    const int n = 5;
    PlacementExample* placementMemory = static_cast<PlacementExample*> (operator new[] (n * sizeof(PlacementExample)));
    for (int i = 0; i < n; ++i) {
        new (placementMemory + i) PlacementExample(rand() % 100 - 200); // No memory is allocated, but memory is initialized
        cout << *(placementMemory + i);
        if (i != n - 1) {
            cout << ' ';
        } else {
            cout << endl;
        }
    }

    cout << endl << endl << "Generating new vector of class PlacementExample from a: " << endl;
    vector <PlacementExample> b;
    cout << "INSRT: ";
    b.insert(b.begin(), a.begin(), a.end());
    print<PlacementExample>(b);

    cout << "EMPLC: ";
    for (int i = 0; i < n; ++i)
    {
        b.emplace(b.begin(), placementMemory[i]);
    }
    print<PlacementExample>(b);

    cout << "ERASE: ";
    b.erase(b.begin());
    print<PlacementExample>(b);

    cout << "ERASE: ";
    b.erase(b.begin() + 3, b.end() - 4);
    print<PlacementExample>(b);

    cout << "EMPLC BACK: ";
    b.emplace_back(*(placementMemory + 3));
    print<PlacementExample>(b);

    cout << "POP BACK: ";
    b.pop_back();
    print<PlacementExample>(b);

    cout << endl << "PlacementExample has no default constructor, since that I use int vector again" << endl;
    cout << "RSZ: ";
    a.resize(5);
    print<int>(a);
    cout << "RSZ: ";
    a.resize(10, 3);
    print<int>(a);

    cout << endl << endl << "Generating new vector of ints of 10 element with value 100: " << endl;
    vector <int> c (10, 100);
    cout << "c: ";
    print<int>(c);
    cout << "a: ";
    print<int>(a);

    cout << "SWP:" << endl;
    a.swap(c);
    cout << "c: ";
    print<int>(c);
    cout << "a: ";
    print<int>(a);
    
    cout << endl <<  "IOTA: ";
    std::iota(a.begin(), a.end(), -4);
    print<int>(a);

    // Deinitialization of memory
    for (int i = 0; i < n; i++)
    {
        placementMemory[i].~PlacementExample();
    }
    operator delete[] (placementMemory);
}

SyntaxError: invalid syntax (<ipython-input-1-ed3d0924279b>, line 9)

### std::array

Used features:
- std::reverse_copy
- exeptions
- templates
- std::ostream_iterator
- std::get

In [2]:
#include <array>
#include <iostream>
#include <stdexcept>

using std::cout;
using std::endl;
using std::array;

template <size_t size>
void print (const array<int,size> &x) {
    for (auto i : x) {
        cout << i << ' ';
    }
    cout << endl;
}
 
int main()
{
    array<int,4> arr {2, 4, 6, 8};
 
    cout << "Second element using []: " << arr[1] << endl;

    cout << "Safe accesory to elements" << endl;
    try {
        for (int i = 0; i < 10; ++i) {
            cout << i << ":";
            cout << arr.at(i) << endl;
        }
    } catch (std::out_of_range) {
        cout << endl;
        cout << "Caught an exception of out of range" << endl;
    }

    std::cout << endl << "Iterating through array: ";
    print <4> (arr);

    cout << endl;
    cout << "BACK: " << arr.back() << " FRONT: " << arr.front() << endl;

    array<int,4> b(arr);

    cout <<"FILL: ";
    arr.fill(5);
    print <4> (arr);

    cout << endl << "SWAP: " << endl;
    cout << "arr: ";
    print <4> (arr);
    cout << "b: ";
    print <4> (b);
    arr.swap(b);
    cout << "arr: ";
    print <4> (arr);
    cout << "b: ";
    print <4> (b);

    cout << endl;
    array<int,10> c{4, 6, 1, -4, 5, 6, 10, 22, 34, -23};
    cout << "c: ";
    print <10> (c);
    cout << "reverse_copy: ";
    std::reverse_copy(c.begin(), c.end(), std::ostream_iterator<int>(cout, " "));
    cout << endl << "SORT: ";
    std::sort(c.begin(), c.end());
    print <10> (c);

    cout << "GET: " << endl;
    std::get<0>(arr) = 1;
    std::get<1>(arr) = 2;
    std::get<2>(arr) = 3;
 
    // get values:
    cout << "arr: ";
    print <4> (arr);

    cout << "[] manipulation: " << endl;
    arr[2] = 4;
    cout << "arr: ";
    print <4> (arr);
}

SyntaxError: invalid syntax (<ipython-input-2-6f6c95d4acb5>, line 5)

### std::deque

Used features:
- ostream overload
- iterator + N

In [9]:
#include <deque>
#include <iostream>

using std::cout;
using std::endl;
using std::deque;

template <typename T>
std::ostream& operator<<(std::ostream& s, const deque<T>& v) 
{
    s.put('[');
    char comma[3] = {'\0', ' ', '\0'};
    for (const auto& e : v) {
        s << comma << e;
        comma[0] = ',';
    }
    return s << ']';
}
 
int main()
{
    // c++11 initializer list syntax:
    deque<std::string> week {"Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"};
    cout << "Week days: " << week << endl;
 
    // words2 == words1
    deque<std::string> words(week.begin(), week.end());
    cout << "words: " << words << endl;

    deque<std::string> words1;
    cout << "words1: " << words1 << endl;
 
    // words2 == words1
    deque<std::string> words2(words);
    cout << "words2: " << words2 << endl;
 
    // words3 is {"Oh", "Ho", "Ho", "Ho", "Ho"}
    deque<std::string> words3(4, "Ho");
    words3.push_front("Oh");
    cout << "words3: " << words3 << endl << endl;

    cout << "EMPTY: words.empty() = " << words.empty() << " words1.empty() = " << words1.empty() << endl;
    cout << "SIZE: " << words.size() << " MAX_SZ: " << words.max_size() << endl << endl;

    words3.pop_front();
    cout << "POP_FR in words3: " << words3 << endl;
    cout << "SIZE: " << words3.size() << " MAX_SZ: " << words3.max_size() << endl;
    words3.clear();
    words3.shrink_to_fit();
    cout << "CLEAR in words3: " << words3 << endl << endl;

    words3.insert(words3.end(), "Ha!");
    cout << "INSERT in words3: " << words3 << endl;
    words3.insert(words3.end(), 4, "He!");
    cout << "INSERT in words3: " << words3 << endl;
    words3.insert(words3.begin(), words2.begin() + 2, words2.end() - 2);
    cout << "INSERT in words3: " << words3 << endl << endl;

    words3.erase(words3.begin() + 2);
    cout << "ERASE in words3: " << words3 << endl;
    words3.erase(words3.begin() + 3, words3.end());
    cout << "ERASE in words3: " << words3 << endl;

    words3.resize(10, "OH Yes!");
    cout << "RESIZE 10 in words3: " << words3 << endl;

    cout << endl << "Swap words: " << words << " words3: " << words3 << endl;
    words.swap(words3);
    cout << "words: " << words << endl << "words3: " << words3 << endl;
}

input_line_23:5:1: error: expected expression
template <typename T>
^


Interpreter Error: 

### std::list

Used:
- prettyprint library
- initializer_list

In [3]:
#include <list>
#include <initializer_list>
#include <iostream>
#include "prettyprint.h"

int main()
{
    std::list <int> l(10, 5);
    std::list <int> l2({10, 5});
    std::cout << l << std::endl << l2 << std::endl;
    l2.assign(l.begin(), l.end());
    std::cout << l2 << std::endl;
    l.unique();
    std::cout << l << std::endl;
    std::list<int> list1 = { 1, 2, 3, 4, 5 };
    std::list<int> list2 = { 10, 20, 30, 40, 50 };
 
    auto it = list1.begin();
    std::advance(it, 2);
 
    list1.splice(it, list2);
 
    std::cout << "list1: " << list1 << "\n";
    std::cout << "list2: " << list2 << "\n";
 
    list2.splice(list2.begin(), list1, it, list1.end());
 
    std::cout << "list1: " << list1 << "\n";
    std::cout << "list2: " << list2 << "\n";

    list1.merge(list2);

    std::cout << "list1: " << list1 << "\n";
    std::cout << "list2: " << list2 << "\n";
}

SyntaxError: invalid syntax (<ipython-input-3-2e6100ece0c9>, line 6)